In [4]:
# The boxes output by the algorithm is based on 416x416 need to rescale back to 640x360
import os
import sys
import glob
import json
import cv2
#output_path = '.'
#pred_path = '.'
pred_path = './test_post_processing/out/'
output_path = './test_post_processing/'
output_img_path = './test_post_processing/out/'
    
# output_path = './predict_416/'
output_name = 'predict_sub.idl'
output_file = os.path.join(output_path,output_name)
out_f = open(output_file, 'w')
img_input_path = './test_post_processing/'
# annotations = os.listdir(pred_path)
annotations = glob.glob(pred_path + '*.json')
# annotations = ['newRight-67564.json']
size = len(annotations)

network_img_sz = 416
img_width = 640
img_height = 360
 
img_plot_flag = 1 # draw boxes on img if value is 1


for i,file in enumerate(annotations):
    sys.stdout.write('\r')
    percentage = 1. * (i+1) / size
    progress = int(percentage * 20)
    bar_arg = [progress*'=', ' '*(19-progress), percentage*100]
    bar_arg += [file]
    sys.stdout.write('[{}>{}]{:.0f}%  {}'.format(*bar_arg))
    sys.stdout.flush()
    
    head, tail = os.path.split(file)
    img_name = os.path.splitext(os.path.basename(file))[0] + '.jpg'
    if img_plot_flag:
        img_file = os.path.join(img_input_path,img_name)
        
        new_img_file = os.path.join(output_img_path,'pred_'+img_name) 
        im = cv2.imread(img_file)
        # im = cv2.resize(im,(network_img_sz,network_img_sz))
    
    # actual parsing
    #file_path = os.path.join(pred_path,file)
    file_path = file
    in_file = open(file_path)
    all_obj = json.load(in_file)
    
    prediction = list()
    if len(all_obj) is 0: # no boxes are predicted
        if img_plot_flag:
            cv2.imwrite(new_img_file, im)
        newLine = {img_name: []}
        out_f.write( json.dumps(newLine) + "\n" )
        continue
    for obj in all_obj:
        label = int(obj['label'])
        confidence = obj['confidence']
        xmax = obj['bottomright']['x']   # these values are already integers
        ymax = obj['bottomright']['y']
        xmin = obj['topleft']['x']
        ymin = obj['topleft']['y']
        newLabels = [xmin, ymin, xmax, ymax, label, confidence]
        # We need to resize boxes to original scale
        xmax_orig = int(xmax/network_img_sz*img_width)
        ymax_orig = int(ymax/network_img_sz*img_height)
        xmin_orig = int(xmin/network_img_sz*img_width)
        ymin_orig = int(ymin/network_img_sz*img_height)
        # newLabels = [xmin_orig, ymin_orig, xmax_orig, ymax_orig, label, confidence]
        prediction.append(newLabels)
        if img_plot_flag:
            cv2.rectangle(im,(xmin,ymin),(xmax,ymax),(255,0,0),2) 
            # cv2.rectangle(im,(xmin_orig,ymin_orig),(xmax_orig,ymax_orig),(255,0,0),2) 
            # cv2.putText(im, str(label), (xmin_orig,ymin_orig-10), cv2.FONT_HERSHEY_DUPLEX, 0.4, (255,0,0), 1)    
    newLine = {img_name: prediction}    
    out_f.write( json.dumps(newLine) + "\n" )
    
    if img_plot_flag:
        cv2.imwrite(new_img_file, im)
        

[====================>]100%  ./test_post_processing/out/60095.json

In [ ]:
#double check labeling result
import cv2
import os
import sys
image_path = './training_aug/'
label_file = './comb-label.idl'
output_path = './training_aug_verification/'
with open(label_file) as f:
    lines = f.readlines() 
    
for i, each in enumerate(lines):
    sys.stdout.write('\r')
    percentage = 1. * (i+1) / size
    progress = int(percentage * 20)
    bar_arg = [progress*'=', ' '*(19-progress), percentage*100]
    bar_arg += [each]
    sys.stdout.write('[{}>{}]{:.0f}%  {}'.format(*bar_arg))
    sys.stdout.flush()
    
    
    line = json.loads(each) 
    image_name = list(line.keys())[0]
    image_file = os.path.join(image_path,image_name)
    new_image_file = os.path.join(output_path,image_name)
    im = cv2.imread(image_file)
    if list(line.values())[0]: #If there is label in annotation
        for box in list(line.values())[0]:        
            xmin = box[0]
            ymin = box[1]
            xmax = box[2]
            ymax = box[3]
            label = box[4]
            cv2.rectangle(im,(int(xmin),int(ymin)),(int(xmax),int(ymax)),(255,0,0),2) 
    cv2.imwrite(new_image_file, im)        

In [5]:
## Given two annotatioin files (one ground truth, one prediction), calculate precision, recall and AP
import os
import sys
import glob
import json
import cv2
import numpy as np
import copy
import math
# w h is input width and height
# W H is output width and height
def convert_to_grid(allobj, W, H, w, h):
    cellx = 1. * w / W
    celly = 1. * h / H
    new_allobj = copy.deepcopy(allobj)
    for obj in new_allobj:
        centerx = .5*(obj[0]+obj[2]) #xmin, xmax
        centery = .5*(obj[1]+obj[3]) #ymin, ymax
        cx = centerx / cellx
        cy = centery / celly
        if cx >= W or cy >= H: return None, None
        obj[2] = float(obj[2]-obj[0]) / cellx
        obj[3] = float(obj[3]-obj[1]) / celly
        #obj[2] = np.sqrt(obj[2])
        #obj[3] = np.sqrt(obj[3])
        obj[0] = cx - np.floor(cx) # centerx
        obj[1] = cy - np.floor(cy) # centery
        obj += [int(np.floor(cy) * W + np.floor(cx))] # index in 1d
    return new_allobj

# these codes are altered from utils/box.py

def overlap(x1,w1,x2,w2):
    l1 = x1 - w1 / 2.;
    l2 = x2 - w2 / 2.;
    left = max(l1, l2)
    r1 = x1 + w1 / 2.;
    r2 = x2 + w2 / 2.;
    right = min(r1, r2)
    return right - left;

# a = [cx, cy, w, h]
def box_intersection(a, b):
    w = overlap(a[0], a[2], b[0], b[2]);
    h = overlap(a[1], a[3], b[1], b[3]);
    if w < 0 or h < 0: return 0;
    area = w * h;
    return area;

def box_union(a, b):
    i = box_intersection(a, b);
    u = a[2] * a[3] + b[2] * b[3] - i;
    return u;

def box_iou(a, b):
    return box_intersection(a, b) / box_union(a, b);

pred_path = './test_post_processing/'
# pred_path = './training_aug/out/'
pred_filename = 'predict_sub.idl'
pred_file = os.path.join(pred_path,pred_filename)
# load prediction into a dictionary
with open(pred_file) as f:
        pred_lines = f.readlines()
pred = dict()
for i, each in enumerate(pred_lines):
    line = json.loads(each)
    img_name = list(line.keys())[0]
    pred_ann = list(line.values())[0]
    pred[img_name] = pred_ann
    
# print(pred)
# load ground truth into dictionary    
grndtrth_path = './test_post_processing/'
grndtrth_filename = 'label_sub.idl'
grndtrth_file = os.path.join(grndtrth_path,grndtrth_filename)    
with open(grndtrth_file) as f:
        grndtrth_lines = f.readlines()
truth = dict()
for i, each in enumerate(grndtrth_lines):
    line = json.loads(each)
    img_name = list(line.keys())[0]
    grndtrth_ann = list(line.values())[0]
    truth[img_name] = grndtrth_ann

# print(truth)
img_input_path = './test_post_processing/'
output_img_path = './test_post_processing/out'
#img_name = os.listdir(img_input_path)
img_file = list(glob.glob(img_input_path + '*.jpg'))

num_boxes = 5
# Use this to get the statistics of number of boxes in one grid
num_boxes_in_grid = [0]*num_boxes
img_plot_flag = 1
num_classes = 4

out_all = []
out_all_label = []
pred_all = []
pred_all_label = []
for i, each_img in enumerate(img_file):
    img_name = os.path.basename(each_img)
    # img = cv2.imread(each_img)   
    # h,w = img.shape
    H = 13
    W = 13
    w = 640
    h = 360
    if img_name not in pred:
        print(img_name + ' not in prediction')
        continue
    if img_name not in truth:
        print(img_name + ' not in ground truth')
        continue
    pred_ann = pred[img_name]
    truth_ann = truth[img_name]
    
    if img_plot_flag:
        img_file = os.path.join(img_input_path,img_name)
        new_img_file = os.path.join(output_img_path,'both_'+img_name) 
        im = cv2.imread(img_file)
        if len(truth_ann) > 0:
            for box in truth_ann:
                cv2.rectangle(im,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(0,255,0),2)
        if len(pred_ann) > 0:
            for box in pred_ann:
                cv2.rectangle(im,(int(box[0]),int(box[1])),(int(box[2]),int(box[3])),(255,0,0),2)        
        cv2.imwrite(new_img_file, im)
        
    pred_grid_ann = convert_to_grid(pred_ann, W, H, w, h )
    truth_grid_ann = convert_to_grid(truth_ann, W, H, w, h)
    # put each annotation into corresponding grids
    
    # Make the assumption that ground truth and prediction must lie in the same grid
    # put prediction into a dictionary, key is the 1d grid location
    # You can also use hash look up to do it. But number of boxes in each image is small, it is fine to just use dictionary
    pred_1d = dict()
    if len(pred_grid_ann) > 0:
        for box in pred_grid_ann:
            cx = box[0] # centerx
            cy = box[1] # centery
            gw = box[2] # sqrt(sw)  sw = (xmax-xmin)/w
            gh = box[3] # sqrt(sh)
            label = box[4]
            conf = box[5]
            pred_all += [conf]
            pred_all_label += [label]
            loc_1d = box[6]
            if loc_1d in pred_1d :
                pred_1d[loc_1d].append([cx, cy, gw, gh, label, conf])
            else:    
                pred_1d[loc_1d] = [[cx, cy, gw, gh, label, conf]]
    
    truth_1d = dict()
    if len(truth_grid_ann) > 0:
        # put ground truth into a dictionary, key is the 1d grid location
        for box in truth_grid_ann:
            cx = box[0] # centerx
            cy = box[1] # centery
            gw = box[2] # sw = (xmax-xmin)/w
            gh = box[3] # 
            label = box[4]
            loc_1d = box[5]
            if loc_1d in truth_1d :
                truth_1d[loc_1d].append([cx, cy, gw, gh, label])
            else:    
                truth_1d[loc_1d] = [[cx, cy, gw, gh, label]]
        out = [- math.inf]* len(truth_grid_ann)  
        out_label = [0] * len(truth_grid_ann) 
        # test number of boxes in one grid   
        box_cnter = 0
        for key, value in truth_1d.items():
            if len(value) > num_boxes:
                print('Exist grid containing more than {} boxes'.format(num_boxes))
            else:
                num_boxes_in_grid[len(value)-1] = num_boxes_in_grid[len(value)-1]+1
            # calculate precision recall and AP
            for each_truth in value: 
                print('Processing box {} of image {}'.format(box_cnter, i))
                out_label[box_cnter] = each_truth[4]
                if key in pred_1d:
                    # calculate iou between ground truth and prediction
                    for each_pred in pred_1d[key]:
                        
                        
                        pred_box = [each_pred[0],each_pred[1],each_pred[2],each_pred[3]]
                        truth_box = [each_truth[0],each_truth[1],each_truth[2],each_truth[3]]
                        print('IOU is {}'.format(box_iou(pred_box, truth_box)))
                        print(pred_box)
                        print(truth_box)
                        print('Label of prediction is {}, Label of ground truth is {}'.format(each_pred[4], each_truth[4]))
                        if box_iou(pred_box, truth_box) > 0.5 and each_pred[4] == each_truth[4]:
                            if out[box_cnter] == -math.inf:
                                out[box_cnter] = each_pred[5] # store the confidence
                            else:
                                out[box_cnter] = -math.inf # the ground truth box corresponds to two predictions
                        
                box_cnter = box_cnter+1    
            
        out_all+= out
        out_all_label+= out_label
        

label_form = [1,2,3,20]        
    # if i is 0: break
out_all_label = np.array(out_all_label)
out_all = np.array(out_all)
pred_all  = np.array(pred_all)
pred_all_label = np.array(pred_all_label)

prec = [0]*len(label_form)
rec = [0]*len(label_form)

# ignore AP calculation for now
for i in range(len(label_form)):
    out_class_idx = out_all_label == label_form[i]
    out_class = out_all[out_class_idx]
    pred_class_idx = pred_all_label == label_form[i]
    pred_class = pred_all[pred_class_idx]
    tp = sum(out_class > 0)
    if len(out_class) > 0:
        rec[i] = tp/len(out_class)
    if len(pred_class) > 0:
        prec[i] = tp/len(pred_class)
        
print(prec)
print(rec)

Processing box 0 of image 0
IOU is 0.9219350247460808
[0.026562499999999822, 0.655555555555555, 3.1687499999999997, 3.7555555555555555]
[0.03502549999999971, 0.7277719999999999, 3.029949, 3.863886]
Label of prediction is 1, Label of ground truth is 1
Processing box 1 of image 0
IOU is 0.6227752574882294
[0.7734375, 0.3305555555555548, 1.05625, 1.0833333333333333]
[0.8259119999999998, 0.5442119999999999, 0.9513139999999995, 1.1495380000000008]
Label of prediction is 1, Label of ground truth is 1
Processing box 0 of image 1
IOU is 0.48377167366383167
[0.27968749999999987, 0.125, 0.8125, 2.022222222222222]
[0.3990190499999997, 0.4861139999999988, 0.6753798999999998, 1.4083420000000006]
Label of prediction is 3, Label of ground truth is 3
Processing box 1 of image 1
IOU is 0.4815098570625801
[0.82265625, 0.5111111111111111, 0.8328125, 1.661111111111111]
[0.9682263500000001, 0.7819429999999992, 1.3203073, 1.300000000000001]
Label of prediction is 1, Label of ground truth is 1
Processing box

In [162]:
truth_grid_ann

[[0.8259119999999998,
  0.5442119999999999,
  0.27051432494416994,
  0.29736509546347245,
  1,
  95],
 [0.03502549999999971,
  0.7277719999999999,
  0.48277634573371553,
  0.5451807039872193,
  1,
  94]]

In [163]:
pred_grid_ann

[[0.03671874999999991,
  0.6916666666666664,
  0.49529031890397374,
  0.5,
  1,
  0.84,
  94],
 [0.8343749999999996,
  0.34861111111111054,
  0.2904737509655563,
  0.29344694769431684,
  1,
  0.71,
  95]]

In [102]:
pred_ann

[[71, 168, 228, 258, 1, 0.84], [211, 188, 265, 219, 1, 0.71]]

In [103]:
truth_ann

[[214.1664, 192.99996, 261.00032, 224.83332000000001, 1],
 [74.83328, 160.49988, 224.0, 267.4998, 1]]

In [118]:
truth_1d 

{94: [[0.03502549999999971,
   0.7277719999999999,
   0.48277634573371553,
   0.5451807039872193,
   1]],
 95: [[0.8259119999999998,
   0.5442119999999999,
   0.27051432494416994,
   0.29736509546347245,
   1]]}

In [119]:
num_boxes_in_grid

[2, 0, 0, 0, 0]

In [123]:
pred_1d

{94: [[0.03671874999999991,
   0.6916666666666664,
   0.49529031890397374,
   0.5,
   1,
   0.84]],
 95: [[0.8343749999999996,
   0.34861111111111054,
   0.2904737509655563,
   0.29344694769431684,
   1,
   0.71]]}

In [115]:
print(a)

[[1]]


In [127]:
import math
out = [-math.inf]* len(truth_grid_ann) 

In [172]:
a = [0.8343749999999996, 0.34861111111111054, 0.084375, 0.08611111111111111]

b = [0.8259119999999998, 0.5442119999999999, 0.07317799999999997, 0.08842600000000007]
box_iou(a,b)
box_intersection(a,b)
overlap(a[0], a[2], b[0], b[2])
overlap(a[1], a[3], b[1], b[3])

-0.10833233333333375

In [154]:

def overlap(x1,w1,x2,w2):
    l1 = x1 - w1 / 2.;
    l2 = x2 - w2 / 2.;
    left = max(l1, l2)
    r1 = x1 + w1 / 2.;
    r2 = x2 + w2 / 2.;
    right = min(r1, r2)
    return right - left;

[0.84, -inf]

In [140]:
print(key in pred_grid_ann)

False


In [151]:
if out[0] is -math.inf:
    print(out)

In [184]:
out_all

[0.84, 0.71, -inf, -inf, 0.85]

In [187]:
out_all_label

[1, 1, 3, 1, 1]

In [191]:
label_form = [1,2,3,20]
out_all_label = np.array(out_all_label)
for i in range(len(label_form)):
    out_class_idx = out_all_label == label_form[i]
    print(out_class_idx)

[ True  True False  True  True]
[False False False False False]
[False False  True False False]
[False False False False False]


In [195]:
out_class_idx = out_all_label == label_form[i]
out_class = out_all[out_class_idx]
si = np.argsort(-out_class)
so = np.sort(-out_class)

In [196]:
out_class

array([0.84, 0.71, -inf, 0.85])

In [205]:
pred_class

0.71

In [206]:
pred_all

array([0.84, 0.71, 0.34, 0.48, 0.32, 0.36, 0.35, 0.59, 0.42, 0.35, 0.85,
       0.44, 0.34])

In [211]:
pred_all_label

array([ 1,  1,  1,  1,  3,  1,  1,  1,  1, 20,  1, 20, 20])

In [212]:
prec

[0.3333333333333333, 0, 0.0, 0.0]

In [213]:
rec

[0.75, 0, 0.0, 0]

In [3]:
annotations

['./test_post_processing/out/60091.json',
 './test_post_processing/out/60092.json',
 './test_post_processing/out/60093.json',
 './test_post_processing/out/60094.json',
 './test_post_processing/out/60095.json']